URLs
- [2017- 2018 Class Size Report City Middle School And High School Core Average Class Size](https://data.cityofnewyork.us/Education/2017-2018-Class-Size-Report-City-Middle-School-And/43qn-d6r8): https://data.cityofnewyork.us/resource/43qn-d6r8.csv?$limit=1000000
- [2010-2011 Class Size - School-level detail](https://data.cityofnewyork.us/Education/2010-2011-Class-Size-School-level-detail/urz7-pzb3)

In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

# read SOCRATA_APP_TOKEN from .env file
load_dotenv()
token = os.getenv("SOCRATA_APP_TOKEN")


In [ ]:

# Define the API endpoint URL
url = "https://data.cityofnewyork.us/api/catalog/v1?limit=10"

# Send a GET request to the API endpoint and get the JSON response
headers = {
    "X-App-Token": token
}

response = requests.get(url, headers=headers)
data = response.json()

results = data["results"]
len(results)

In [38]:
datasets = {result["metadata"]["domain"]: result for result in results}
datasets.keys()

dict_keys(['www.dallasopendata.com', 'data.ny.gov', 'data.ct.gov', 'data.cdc.gov', 'opendata.howardcountymd.gov', 'data.cityofchicago.org', 'data.nj.gov', 'data.wa.gov', 'data.cityofnewyork.us', 'data.nashville.gov', 'data.memphistn.gov', 'www.datos.gov.co', 'healthdata.gov', 'health.data.ny.gov', 'data.mo.gov', 'datahub.transportation.gov', 'data.montgomerycountymd.gov', 'data.fortworthtexas.gov', 'data.pa.gov', 'data.kingcounty.gov', 'data.lacity.org', 'data.colorado.gov', 'data.virginia.gov', 'data.calgary.ca', 'data.cambridgema.gov', 'data.providenceri.gov', 'data.sfgov.org', 'insights.cincinnati-oh.gov', 'data.pr.gov', 'data.delaware.gov', 'priv-data.ojp.usdoj.gov', 'datahub.usac.org', 'data.vermont.gov', 'data.bts.gov', 'datacatalog.cookcountyil.gov', 'data.cincinnati-oh.gov', 'data.oregon.gov', 'data.lacounty.gov', 'data.edd.ca.gov', 'data.energystar.gov', 'data.norfolk.gov', 'data.hawaii.gov', 'data.cityofgainesville.org', 'data.hartford.gov', 'internal.chattadata.org', 'intern

In [ ]:

# do a dict comprehension to add each item in results keyed by metadata.domain
datasets = {result["metadata"]["domain"]: result for result in results}


# Extract the intersecting keys from the results dictionary
keys = set(results[0].keys())
for result in results:
    keys &= set(result.keys())

# Create a list of dictionaries with only the intersecting keys
filtered_results = []
for result in results:
    filtered_result = {}
    for key in keys:
        filtered_result[key] = result[key]
    filtered_results.append(filtered_result)

# Create a DataFrame from the list of dictionaries
# df = pd.DataFrame(filtered_results)
# df.head()
datasets.keys()
len(results)

In [ ]:
import pandas as pd

# meta = "https://data.cityofnewyork.us/api/views/metadata/v1"
# meta = "https://data.cityofnewyork.us/api/catalog/v1.csv?ids=*"
# get a list of all of the data sets in cityofnewyork socrata
meta = "https://data.cityofnewyork.us/api/catalog/v1?domains=data.cityofnewyork.us"
df = pd.read_json(meta, lines=True)
meta.head()

In [20]:

{
    "doe_22": "https://infohub.nyced.org/docs/default-source/default-document-library/updated2023_avg_classsize_schl.xlsx",

}

url = "https://data.cityofnewyork.us/resource/sgr7-hhwp.csv?$limit=1000000"

# nyc = pd.read_csv(url)
# print(df.columns)
# df.grade_level.unique()
# df.head()

url = "https://infohub.nyced.org/docs/default-source/default-document-library/updated2023_avg_classsize_schl.xlsx"
xls = pd.read_excel(url, sheet_name=None)
# first sheet is k-8
k8 = xls["K-8 Avg"]
hs = xls["MS HS Avg"]
ptr = xls["PTR"]
k8.head()

/home/mxc/.virtualenvs/school-data/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/mxc/.virtualenvs/school-data/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/mxc/.virtualenvs/school-data/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,DBN,School Name,Grade Level,Program Type,Number of Students,Number of Classes,Average Class Size,Minimum Class Size,Maximum Class Size
0,01M015,PS 015 ROBERTO CLEMENTE,K,G&T,11,1,11.0,<15,<15
1,01M015,PS 015 ROBERTO CLEMENTE,K,ICT,25,1,25.0,25,25
2,01M015,PS 015 ROBERTO CLEMENTE,1,G&T,14,1,14.0,<15,<15
3,01M015,PS 015 ROBERTO CLEMENTE,1,ICT,17,1,17.0,17,17
4,01M015,PS 015 ROBERTO CLEMENTE,2,G&T,7,1,7.0,<15,<15


In [17]:
# show the intersecting columns
print(set(k8.columns) - set(hs.columns))
print(set(hs.columns) - set(k8.columns))
print(len(hs.columns) , len(k8.columns))

set()
{'Subject', 'Department'}
11 9


In [23]:
k8["Subject"] = "K8"
k8["Department"] = "K8"
df = pd.concat([k8, hs], axis=0).head()
df["ay"] = 2022
cols = {
    "DBN": "dbn",
    "School Name": "school_name",
    "Grade Level": "grade",
    "Program Type": "program_type",
    "Number of Students": "students_n",
    "Number of Classes": "classes_n",
    "Average Class Size": "avg_class_size",
    "Subject": "subject",
    "Department": "dpt",
    "Minimum Class Size": "min_class_size",
    "Maximum Class Size": "max_class_size"
}
df = df.rename(columns=cols)
# drop school_name
df = df.drop(columns=["school_name"])
df.head()
df.to_csv("/opt/nycschools/nyc_class_size.csv", index=False)

In [5]:
df = pd.read_csv("https://data.cityofnewyork.us/resource/43qn-d6r8.csv?$limit=1000000")
print(df.columns)
df

Index(['grade_level', 'program_type', 'department', 'subject',
       'number_of_students', 'number_of_classes', 'average_class_size'],
      dtype='object')


,grade_level,program_type,department,subject,number_of_students,number_of_classes,average_class_size
0,HS Core,Gen Ed,English,HS English 09,48093,1897,25.4
1,HS Core,Gen Ed,English,HS English 10,46274,1762,26.3
2,HS Core,Gen Ed,English,HS English 11,45892,1780,25.8
3,HS Core,Gen Ed,English,HS English 12,44382,1735,25.6
4,HS Core,Gen Ed,English,HS English Other,75065,3458,21.7
...,...,...,...,...,...,...,...
95,MS Core,SC,Math,MS Math Other,598,60,10.0
96,MS Core,SC,Science,MS Science Core,10747,1095,9.8
97,MS Core,SC,Science,MS Science Other,210,23,9.1
98,MS Core,SC,Social Studies,MS Social Studies Core,10663,1079,9.9
